In [ ]:
!pip install tensorflow

In [ ]:
!nvidia-smi

In [ ]:
!pip install tensorflow-gpu


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
from transformers import TFBertModel


# model creation--------------------------------------------------------------- 

In [7]:
df = pd.read_csv('/content/drive/MyDrive/FYPdataset/Output_Dataset final 4.3.2023 tensorflow ready.csv', delimiter=",")
df.head()

,Unnamed: 0,ISBN,Book_Title,Description,sentiment_score_neg,sentiment_score_neu,sentiment_score_pos,sentiment_score_compound,sentiment_type
0,0,0393045218,The Mummies of Ürümchi,A look at the incredibly well-preserved ancien...,0.000,1.000,0.000,0.0000,2
1,1,0399135782,The Kitchen God's Wife,Here is the eagerly awaited new novel from the...,0.014,0.693,0.293,0.9666,1
2,2,0425176428,What If?,"Essays by respected military historians, inclu...",0.064,0.871,0.064,0.0000,1
3,3,0679425608,Under the Black Flag,"For this rousing, revisionist history, the for...",0.000,0.898,0.102,0.6908,1
4,4,074322678X,Where You'll Find Me,"Now back in print, Ann Beattie's finest short ...",0.092,0.776,0.132,0.5423,1


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df['sentiment_type'].value_counts()

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [9]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [10]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Description'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [11]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [12]:
labels = np.zeros((len(df), 3))

In [13]:
labels[np.arange(len(df)),df["sentiment_type"].values]=1

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))


In [15]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [16]:
dataset = dataset.map(SentimentDatasetMapFunction)  # converting to required format for tensorflow dataset



In [17]:


dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)  # batch size, drop any left out tensor

In [18]:
train_size = int(
    (len(df) // 16) * 0.8)  # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [ ]:
train_size

7376

In [19]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
val_dataset2 =  tf.keras.utils.to_categorical(train_size)

# model------------------------------------------------------------------------------------------------------------------

In [ ]:
model = TFBertModel.from_pretrained('bert-base-cased')  # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

In [ ]:
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] 
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) 

In [ ]:
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
print(sentiment_model.summary())

In [ ]:
optim = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="Adam",
)

loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])


hist = sentiment_model.fit(train_dataset,validation_data=val_dataset,epochs=1)

sentiment_model.save('/content/drive/MyDrive/FYPdataset/sentiment_modelnew')



# **Testing-------------------------------------------------------------------------**

In [ ]:
from sklearn.metrics import classification_report
# Train your model and get predictions for the validation set
model = tf.keras.models.load_model('/content/drive/MyDrive/FYPdataset/sentiment_modelnew')
y_pred = model.predict(train_dataset)


7376/7376 [==============================] - 1993s 270ms/step


In [21]:
from sklearn.metrics import classification_report
# Train your model and get predictions for the validation set
model = tf.keras.models.load_model('/content/drive/MyDrive/FYPdataset/sentiment_modelnew')
y_pred2 = model.predict(val_dataset)

   3/1844 [..............................] - ETA: 13:01:17

KeyboardInterrupt: ignored

In [ ]:
y_pred2

In [ ]:
y_true = []
for _, labels in val_dataset:
    y_true.extend(np.argmax(labels.numpy(), axis=1))

# Convert the predicted probabilities to class labels (0, 1, or 2)
y_pred2 = np.argmax(y_pred2, axis=1)

# Calculate and print the classification report, which includes precision, recall, and F1 metrics
print(classification_report(y_true, y_pred2))

# **Prediction maker-------------------------------------------------------------**

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer
from transformers import TFBertModel

# Load the saved model
sentiment_model = tf.keras.models.load_model('/content/drive/MyDrive/FYPdataset/sentiment_modelnew')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Prepare the input text data
text = "This is a positive sentence."
tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )

# Make a prediction
prediction = sentiment_model.predict({'input_ids': tokenized_text['input_ids'], 'attention_mask': tokenized_text['attention_mask']})
print(prediction)

1/1 [==============================] - 3s 3s/step
[[0.00090257 0.89328414 0.10581322]]


# **----------------------------------------------------------------------------**

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score

# Load the iris dataset
data = load_iris()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Encode the target variable
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

# Step 2: Preprocess the dataset

# Step 3: Build a neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Step 4: Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=16)

# Step 5: Evaluate the model
y_predit = model.predict(X_test)
y_pred_classes = tf.argmax(y_pred, axis=1)
y_test_classes = tf.argmax(y_test, axis=1)

f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')

print(f'F1 score: {f1}')
print(f'Recall score: {recall}')
print(f'Precision score: {precision}')
print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score


y_pred_classes = tf.argmax(y_pred, axis=1)
y_test_classes = tf.argmax(y_test, axis=1)

f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')

print(f'F1 score: {f1}')
print(f'Recall score: {recall}')
print(f'Precision score: {precision}')
print(classification_report(y_test_classes, y_pred_classes))